## Workshop TP3

## 1) EDA

## 2) Defino X, Y 

## 3) Split Train, Test (Utilizar shuffle = True)

## 4) Derivo las variables (dummies/one hot encoder, scaling)

## 5) Instancio el modelo (con train)

## 6) Entrenar el modelo

## 7) Performance

**Test**

Luego tenés que escalar y transformar el test

In [130]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn import metrics
import statsmodels.api as sm
from statsmodels.tools import eval_measures

In [131]:
data_CABA = pd.read_csv('..\properati_CABA.csv', sep=",")

In [132]:
data_CABA.rename(columns = {'price_m2_calc_supTotal': 'price_m2', 'surface_total_in_m2_CLEAN': 'surface'}, inplace = True)

In [133]:
data_CABA.head()

,Unnamed: 0,property_type,place_name,place_with_parent_names,state_name,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,...,contrafrente,cochera,amenities,esp_exterior,mas_1banio,estrenar,surface,bin_price,bin_sup,price_m2
0,0,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Capital Federal,-34.661824,-58.508839,62000.0,55.0,40.0,...,0,0,0,1,1,0,55.0,"(19999.999, 83000.0]","(48.0, 68.0]",1127.272727
1,1,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Bs.As. G.B.A. Zona Sur,-34.903883,-57.964330,150000.0,NaN,NaN,...,0,1,1,1,1,1,80.0,"(120000.0, 170000.0]","(68.0, 122.0]",1875.000000
2,2,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Capital Federal,-34.652262,-58.522982,72000.0,55.0,55.0,...,0,0,0,0,1,0,55.0,"(19999.999, 83000.0]","(48.0, 68.0]",1309.090909
3,3,PH,Liniers,|Argentina|Capital Federal|Liniers|,Capital Federal,-34.647797,-58.516424,95000.0,NaN,NaN,...,0,0,0,1,0,0,57.5,"(83000.0, 120000.0]","(48.0, 68.0]",1652.173913
4,4,apartment,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Buenos Aires Costa Atlántica,-38.002626,-57.549447,64000.0,35.0,35.0,...,1,0,0,0,1,0,35.0,"(19999.999, 83000.0]","(5.999, 48.0]",1828.571429


#### Definimos X e Y

In [134]:
X = data_CABA.drop(['price_aprox_usd', 'price_m2', 'state_name'], axis = 1)
y = data_CABA.price_m2

In [135]:
data_CABA.shape

(107708, 23)

In [136]:
y = data_CABA['price_m2']
X = data_CABA[['surface', 'place_name', 'property_type']]

#### Dividimos en Train y Test

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 13)

In [138]:
data_CABA[['price_m2','surface', 'place_name', 'property_type']].corr()

,price_m2,surface
price_m2,1.000000,-0.146424
surface,-0.146424,1.000000


#### Obtenemos Dummies

In [139]:
X_train_dummies = pd.get_dummies(X_train, drop_first = True).reset_index(drop = True)

In [140]:
y_train = y_train.reset_index(drop = True)

#### Version alternativa OneHot

In [141]:
from sklearn.preprocessing import OneHotEncoder 
onehot_encoder2 = OneHotEncoder(drop='first')
dummy_oneHot_correct = onehot_encoder2.fit_transform(X_train[['place_name', 'property_type']])
dummy_oneHot_correct = pd.DataFrame(dummy_oneHot_correct.toarray())
dummy_oneHot_correct.head()

,0,1,2,3,4,5,6,7,8,9,...,221,222,223,224,225,226,227,228,229,230
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Stats Model

In [142]:
X_stats = sm.add_constant(X_train_dummies)

model = sm.OLS(y_train, X_stats).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               price_m2   R-squared:                       0.706
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     890.6
Date:                Tue, 24 Aug 2021   Prob (F-statistic):               0.00
Time:                        15:38:43   Log-Likelihood:            -6.5788e+05
No. Observations:               86166   AIC:                         1.316e+06
Df Residuals:                   85933   BIC:                         1.318e+06
Df Model:                         232                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
const                                    1773.8591     53.350     33.249      0.000    1669.293    1878.425
surface                                    -0.0534      0.002    -22.931      0.000      -0.058      -0.049
place_name_Agronomía                     -176.7513     97.924     -1.805      0.071    -368.681      15.178
place_name_Aguas Verdes                 -1505.3450    160.158     -9.399      0.000   -1819.254   -1191.436
place_name_Almagro                       -171.2175     55.445     -3.088      0.002    -279.889     -62.546
place_name_Almirante Brown               -522.6084     54.848     -9.528      0.000    -630.109    -415.108
place_name_Alta Gracia                  -1349.2290    154.177     -8.751      0.000   -1651.415   -1047.043
place_name_Avellaneda                    -739.8335     57.470    -12.873      0.000    -852.474    -627.193
place_name_Azul                         -1362.2510    167.157     -8.150      0.000   -1689.877   -1034.625
place_name_Bahía Blanca                  -912.0830     58.842    -15.501      0.000   -1027.413    -796.753
place_name_Balcarce                     -1227.7261     78.453    -15.649      0.000   -1381.494   -1073.958
place_name_Balvanera                     -457.9290     57.152     -8.013      0.000    -569.946    -345.912
place_name_Barracas                      -404.6065     59.687     -6.779      0.000    -521.592    -287.621
place_name_Barrio Norte                   474.8252     55.549      8.548      0.000     365.949     583.701
place_name_Belgrano                       574.7406     53.918     10.660      0.000     469.062     680.419
place_name_Berazategui                   -230.5162     58.860     -3.916      0.000    -345.881    -115.152
place_name_Berisso                      -1015.4120    139.870     -7.260      0.000   -1289.556    -741.268
place_name_Boca                          -683.1554     64.795    -10.543      0.000    -810.153    -556.157
place_name_Boedo                         -238.9993     56.624     -4.221      0.000    -349.982    -128.016
place_name_Brandsen                      -216.6168    110.667     -1.957      0.050    -433.523       0.290
place_name_Caballito                       63.3468     54.174      1.169      0.242     -42.833     169.527
place_name_Caleta Olivia                 -999.6168    167.182     -5.979      0.000   -1327.291    -671.942
place_name_Campana                      -1247.9282    136.108     -9.169      0.000   -1514.699    -981.158
place_name_Capitán Bermúdez             -1161.2628     97.076    -11.962      0.000   -1351.531    -970.994
place_name_Cariló                        -353.0211     91.600     -3.854      0.000    -532.555    -173.487
place_name_Cañuelas                     -1005.8634    100.0

### Tiro los que no pasan el test "p.value"

In [143]:
X_train_dummies_pvalue = X_train_dummies.drop(["place_name_Villa General Mitre","place_name_Agronomía","place_name_Almagro","place_name_Chacarita","place_name_Centro / Microcentro","place_name_Monte Castro","place_name_Monserrat","place_name_Parque Centenario","place_name_Parque Chas","place_name_San Nicolás","place_name_San Telmo","place_name_Versalles","place_name_Villa Devoto","place_name_Villa Ortuzar","place_name_Villa Pueyrredón","place_name_Villa Santa Rita","place_name_Villa del Parque","property_type_house"], axis=1)

In [144]:
X_stats = sm.add_constant(X_train_dummies_pvalue)

model = sm.OLS(y_train, X_stats).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               price_m2   R-squared:                       0.703
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     950.6
Date:                Tue, 24 Aug 2021   Prob (F-statistic):               0.00
Time:                        15:38:45   Log-Likelihood:            -6.5836e+05
No. Observations:               86166   AIC:                         1.317e+06
Df Residuals:                   85951   BIC:                         1.319e+06
Df Model:                         214                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
const                                    1461.7425      8.770    166.667      0.000    1444.552    1478.932
surface                                    -0.0573      0.002    -24.486      0.000      -0.062      -0.053
place_name_Aguas Verdes                 -1379.1822    152.224     -9.060      0.000   -1677.540   -1080.824
place_name_Almirante Brown               -446.7909     16.714    -26.732      0.000    -479.550    -414.032
place_name_Alta Gracia                  -1284.8369    145.835     -8.810      0.000   -1570.673    -999.001
place_name_Avellaneda                    -627.6359     24.059    -26.087      0.000    -674.792    -580.480
place_name_Azul                         -1289.0018    159.641     -8.074      0.000   -1601.898    -976.106
place_name_Bahía Blanca                  -821.3198     27.186    -30.211      0.000    -874.604    -768.036
place_name_Balcarce                     -1157.9322     58.820    -19.686      0.000   -1273.219   -1042.646
place_name_Balvanera                     -341.8060     23.274    -14.686      0.000    -387.423    -296.189
place_name_Barracas                      -287.6950     29.003     -9.919      0.000    -344.542    -230.848
place_name_Barrio Norte                   580.7062     18.938     30.664      0.000     543.588     617.824
place_name_Belgrano                       679.8959     13.339     50.969      0.000     653.751     706.041
place_name_Berazategui                   -157.7210     27.199     -5.799      0.000    -211.032    -104.411
place_name_Berisso                       -947.3967    130.442     -7.263      0.000   -1203.063    -691.731
place_name_Boca                          -571.4417     38.521    -14.835      0.000    -646.942    -495.941
place_name_Boedo                         -129.3032     21.924     -5.898      0.000    -172.274     -86.333
place_name_Brandsen                      -139.1553     98.076     -1.419      0.156    -331.384      53.073
place_name_Caballito                      172.9364     14.354     12.048      0.000     144.803     201.070
place_name_Caleta Olivia                 -940.7943    159.663     -5.892      0.000   -1253.733    -627.856
place_name_Campana                      -1186.0674    126.352     -9.387      0.000   -1433.716    -938.418
place_name_Capitán Bermúdez             -1100.1727     82.240    -13.378      0.000   -1261.362    -938.983
place_name_Cariló                        -282.6893     75.628     -3.738      0.000    -430.920    -134.459
place_name_Cañuelas                      -943.9049     85.711    -11.013      0.000   -1111.899    -775.911
place_name_Chascomús                     -749.2680     79.198     -9.461      0.000    -904.496    -594.040
place_name_Cipolletti                    -807.1566    126.2

#### Entreno Lasso

In [145]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

In [146]:
x_train_sup_sc = sc.fit_transform(X_train_dummies_pvalue['surface'].values.reshape(-1,1))

In [147]:
x_train_sup_sc = pd.DataFrame(x_train_sup_sc, columns = ['surface'])

In [148]:
X_train_dummies['surface'] = x_train_sup_sc

In [149]:
# Las dummies no hace falta que se reescalen

In [150]:
import numpy as np

alphas = np.linspace(0.01, 100, 50) #Aquí busco el hiperparámetro


In [151]:
model = linear_model.LassoCV(alphas = alphas, cv = 5)

In [152]:
cv = KFold(5, shuffle=True)
lassocv = linear_model.LassoCV(alphas=np.linspace(0.001,1, 100), cv=cv, normalize=True)
lassocv.fit(X_train_dummies, y_train)
predictions = lassocv.predict(X_train_dummies)

print ("\nr^2:", lassocv.score(X_train_dummies, y_train))
print ("\nalpha:", lassocv.alpha_)
print ("\ncoeficientes:", lassocv.coef_)
print ("\nintercepto:", lassocv.intercept_)
print ("\nzero_coefs:", (lassocv.coef_ == 0).sum())


r^2: 0.7059954644269322

alpha: 0.001

coeficientes: [-5.05256182e+01  2.56384071e+02 -1.03123960e+03  2.73360885e+02
 -7.07191697e+01 -8.74363582e+02 -2.88028683e+02 -8.85728660e+02
 -4.59259129e+02 -7.68265540e+02 -6.78201873e+00  3.82688898e+01
  9.19376423e+02  1.02047255e+03  2.14355553e+02 -5.43607065e+02
 -2.29052990e+02  2.05063117e+02  2.15935260e+02  5.08806542e+02
 -5.22271975e+02 -7.76345600e+02 -6.97328448e+02  8.36143101e+01
 -5.41688542e+02  2.97209856e+02  3.17658311e+02 -3.82561927e+02
 -4.21614120e+02 -7.39143161e+02  6.68398233e+02  6.55256193e+02
 -8.37286299e+02 -3.08194345e+02  6.57349930e+01 -6.15255081e+02
  3.85888950e+01 -5.89312991e+01 -1.19192493e+02 -8.90846792e+02
 -6.71042883e+02 -7.54662301e+02 -6.20532223e+02  2.04257030e+02
 -1.15670368e+02  3.60687594e+00 -5.57747730e+02 -0.00000000e+00
  1.78856476e+02 -5.39136255e+01  1.85529798e+03  0.00000000e+00
 -4.92169838e+02  2.00771116e+01 -3.86945639e+02  2.04217491e+02
 -4.04491357e+01 -4.15268824e+02 -1.

In [153]:
pd.DataFrame([[v, c] for v,c in zip(X_train_dummies.columns, lassocv.coef_)]).head()

,0,1
0,surface,-50.525618
1,place_name_Agronomía,256.384071
2,place_name_Aguas Verdes,-1031.239599
3,place_name_Almagro,273.360885
4,place_name_Almirante Brown,-70.719170


#### Regularizo a Stats Models para ver quecomo queda

In [154]:
best_alpha = lassocv.alpha_

no_reg_model = sm.OLS(y_train, X_stats)

reg_model = no_reg_model.fit_regularized(alpha = best_alpha, L1_wt = 1)

In [155]:
predictions = reg_model.predict(X_stats)

In [156]:
#metrics.r2_score(X_stats, predictions)

#### Entreno RidgeCV

In [157]:
cv = KFold(5, shuffle=True)
RidgeCV = linear_model.RidgeCV(alphas=np.linspace(0.001,1, 100), cv=cv, normalize=True)
RidgeCV.fit(X_train_dummies, y_train)
predictions = RidgeCV.predict(X_train_dummies)

print ("\n r^2:", RidgeCV.score(X_train_dummies, y_train))
print ("\n alpha:", RidgeCV.alpha_)
print ("\n coeficientes:", RidgeCV.coef_)
print ("\n intercepto:", RidgeCV.intercept_)


 r^2: 0.7062294369673436

 alpha: 0.001

 coeficientes: [-5.05517204e+01 -9.32644595e+00 -1.33655933e+03 -3.49423333e+00
 -3.54398638e+02 -1.18015699e+03 -5.71589652e+02 -1.19313488e+03
 -7.43476441e+02 -1.05874411e+03 -2.89914578e+02 -2.36697660e+02
  6.42042227e+02  7.41866346e+02 -6.25255013e+01 -8.46627647e+02
 -5.14907243e+02 -7.11414721e+01 -4.87696958e+01  2.30921609e+02
 -8.30832787e+02 -1.07892964e+03 -9.92320061e+02 -1.84914380e+02
 -8.37057665e+02  2.43715263e+01  4.45040388e+01 -6.76978432e+02
 -7.23106501e+02 -1.04511334e+03  3.94188180e+02  3.80236060e+02
 -1.14425034e+03 -5.98686939e+02 -1.86634379e+02 -9.14878424e+02
 -2.35007162e+02 -3.43868689e+02 -4.05426681e+02 -1.18938695e+03
 -9.62738905e+02 -1.06830307e+03 -9.28732488e+02 -6.81318369e+01
 -4.22268983e+02 -2.68110194e+02 -8.38621188e+02 -2.83198750e+02
 -8.77264484e+01 -3.37764873e+02  1.59879110e+03 -2.79152367e+02
 -7.94387556e+02 -2.56603266e+02 -6.83395555e+02 -7.29329074e+01
 -3.24490106e+02 -7.09574416e+02 

In [158]:
#print ("\nr^2 test:", RidgeCV.score(X_test, y_test))

# me arroja "could not convert string to float: 'Recoleta'"  ??????????????????????????????

#### Elastic Net

In [159]:
elastic_net = linear_model.ElasticNet(alpha=0.011, normalize=True)
elastic_net.fit(X_train_dummies, y_train)
predictions = elastic_net.predict(X_train_dummies)

print ("\n r^2:", elastic_net.score(X_train_dummies, y_train))
print ("\n coeficientes:", elastic_net.coef_)
print ("\n intercepto:", elastic_net.intercept_)


 r^2: 0.00508936321977338

 coeficientes: [-2.75941122e-01  5.62292444e-01 -2.35615405e+00  9.00598844e-01
 -1.10963815e+00 -3.13354940e+00 -6.30871054e-01 -2.43511034e+00
 -1.15805486e+00 -2.51554621e+00  3.00370753e-01  1.95886355e-01
  2.37818147e+00  2.59165272e+00 -2.98959859e-01 -1.92908135e+00
 -2.33216042e-01  7.42553857e-01 -1.33765066e+00  1.47656985e+00
 -2.20759973e+00 -2.88526268e+00 -2.60083830e+00 -6.39842814e-01
 -2.33541817e+00  8.82240592e-01  9.38793270e-01 -1.76916304e+00
 -1.00378361e+00 -2.69662539e+00  1.60826469e+00  1.55461967e+00
 -2.94699195e+00 -1.34160035e+00 -2.07730384e-01 -1.48892791e+00
 -1.17536313e-01  2.44420960e-01  0.00000000e+00 -3.08712937e+00
 -2.40522072e+00 -2.65984789e+00 -1.14446307e+00 -7.99927971e-01
 -1.36331896e+00 -8.85775869e-01 -1.43169191e+00 -1.07819802e+00
 -1.67012160e-03 -1.33116838e+00  2.30582568e+00 -1.01130335e+00
 -2.34018749e+00 -4.88974050e-01 -1.51251772e+00  6.36313923e-01
 -0.00000000e+00 -2.17797477e-01 -1.59136138e+0

In [160]:
from sklearn.metrics import r2_score

In [161]:
# Hacer funciones que reescalen

In [162]:
# Hacer un DataFrame que compare los distintos modelos 

#### Probando con Surface de distintos grados  (da mejroes resultados)

In [163]:
X_train_dummies["surface2"] = X_train_dummies.surface * X_train_dummies.surface

cv = KFold(5, shuffle=True)
lassocv = linear_model.LassoCV(alphas=np.linspace(0.001,1, 1000), cv=cv, normalize=True)
lassocv.fit(X_train_dummies, y_train)
predictions = lassocv.predict(X_train_dummies)

print ("\nr^2:", lassocv.score(X_train_dummies, y_train))
print ("\nalpha:", lassocv.alpha_)
print ("\nzero_coefs:", (lassocv.coef_ == 0).sum())


r^2: 0.7155466215840498

alpha: 0.001

zero_coefs: 3


In [ ]:
X_train_dummies["surface2"] = X_train_dummies.surface * X_train_dummies.surface
X_train_dummies["surface3"] = X_train_dummies.surface2 * X_train_dummies.surface 

cv = KFold(5, shuffle=True)
lassocv = linear_model.LassoCV(alphas=np.linspace(0.001,1, 1000), cv=cv, normalize=True)
lassocv.fit(X_train_dummies, y_train)
predictions = lassocv.predict(X_train_dummies)

print ("\nr^2:", lassocv.score(X_train_dummies, y_train))
print ("\nalpha:", lassocv.alpha_)
print ("\nzero_coefs:", (lassocv.coef_ == 0).sum())

In [ ]:
X_train_dummies["surface2"] = X_train_dummies.surface * X_train_dummies.surface
X_train_dummies["surface3"] = X_train_dummies.surface2 * X_train_dummies.surface 
X_train_dummies["surface4"] = X_train_dummies.surface3 * X_train_dummies.surface 

cv = KFold(5, shuffle=True)
lassocv = linear_model.LassoCV(alphas=np.linspace(0.001,1, 1000), cv=cv, normalize=True)
lassocv.fit(X_train_dummies, y_train)
predictions = lassocv.predict(X_train_dummies)

print ("\nr^2:", lassocv.score(X_train_dummies, y_train))
print ("\nalpha:", lassocv.alpha_)
print ("\nzero_coefs:", (lassocv.coef_ == 0).sum())